# snapReactors

Copyright (c) Dan Kotlyar and CoRE group

# Materials Container

* This container stores materials information such as isotopic definition, abundances, uncertainties and material properties.
* The container also stores, modifies or creates new h5 files for data storage.

## Code: 

In [5]:
import os as os
import numpy as np
from snapReactors.containers.materials import Material, Composition

## Defining a new material

### Define with Materials container

1. Give name of material
2. Give type of uncertainty, must be in ``Enum.UTYPE``
3. Give the composition type, must be in ``Enum.CTYPE``
4. Give the isotopes that define a material
5. Give the abundances for each isotope
6. The uncertainty value, temperature and pressure dependencies, reference, description and filename to read data from are left as optional parameters.
    1. Note that temperature and pressure dependencies must be under ``ALLOWED_PROPERTIES``


In [6]:
mat1 = Material("newMat", 'NONE', 'WEIGHT', np.array([]), np.array([]), None, np.array([300, 900, 1800]), np.array([10E+6, 11E+6]), reference=None, description='This is an example')
print(mat1)

{'matName': ['newMat'], 'utype': [<UTYPE.NONE: 4>], 'ctype': [<CTYPE.WEIGHT: 2>], 'abundances': [array([], dtype=float64)], 'isotopes': [array([], dtype=float64)], 'unc': [None], 'temperatures': [array([ 300,  900, 1800])], 'pressures': [array([10000000., 11000000.])], 'reference': [None], 'description': ['This is an example'], '_properties': []}


### Using the Composition subcontainer

The goal of the ``Composition`` subcontainer is to provide an easier interface. As such this effectively excludes temperature and pressure dependencies.
1. Give name of material
2. Give type of uncertainty, must be in ``Enum.UTYPE``
3. Give the composition type, must be in ``Enum.CTYPE``
4. Give the isotopes that define a material
5. Give the abundances for each isotopes

In [7]:
mat2 = Composition('newmat', 'NONE', 'WEIGHT', np.array([]), np.array([]), description='This example uses the Composition subcontainer')
print(mat2)

{'matName': ['newmat'], 'utype': [<UTYPE.NONE: 4>], 'ctype': [<CTYPE.WEIGHT: 2>], 'abundances': [array([], dtype=float64)], 'isotopes': [array([], dtype=float64)], 'unc': [None], 'temperatures': [None], 'pressures': [None], 'reference': [None], 'description': ['This example uses the Composition subcontainer'], '_properties': []}


### Reading in isotopic defintiion through a text file

1. The ``Materials.readData(filename)`` method is used to read in all relevenat materials information through a text file. 
2. The text file follows a specific structure that pulls in all parameters specific to the material.
    1. All parameters must be included in .txt file except for property parameters


####  Example text file shown below

In [8]:
#text_file = open('test.txt')
text_file = open('c:\\Users\\Samuel\\Documents\\GitHub\\SNAP-REACTORS\\snapReactors\\containers\\test.txt')
file_content = text_file.read()
print(file_content)
text_file.close()

Material Name: hasteC
ctype: ATOMIC
utype: NONE
Number of isotopes: 33
Isotopic Definition:
--------------------
6000.03c     -0.0007    
27059.03c    -0.0125    
24050.03c    -0.006952
24052.03c    -0.1340624
24053.03c    -0.0152016
24054.03c    -0.003784
42092.03c    -0.0249033
42094.03c    -0.0156179
42095.03c    -0.0269841
42096.03c    -0.0283441
42097.03c    -0.0162894
42098.03c    -0.0412964
42100.03c    -0.0165648
23050.03c    -0.0000075
23051.03c    -0.0029925
74180.03c    -0.00048
74182.03c    -0.106
74183.03c    -0.05724
74184.03c    -0.12256
74186.03c    -0.11372
26054.03c    -0.003360875
26056.03c    -0.05275855
26057.03c    -0.001218425
26058.03c    -0.00016215
25055.03c    -0.01     
14028.03c     -0.00645561
14029.03c     -0.00032795
14030.03c     -0.00032795
28058.03c     -0.1220600887
28060.03c     -0.0470180183
28061.03c     -0.0020438407
28062.03c     -0.0065166585
28064.03c     -0.0016596008
Properties: [
]
Reference: NA-Examples
Description: This is an example inpu

#### Materials definition returned by readData

In [9]:
Material.readData(mat1, 'c:\\Users\\Samuel\\Documents\\GitHub\\SNAP-REACTORS\\snapReactors\\containers\\test.txt')
print(mat1)

{'matName': ['newMat', ' hasteC\n'], 'utype': [<UTYPE.NONE: 4>, <UTYPE.NONE: 4>], 'ctype': [<CTYPE.WEIGHT: 2>, <CTYPE.ATOMIC: 1>], 'abundances': [array([], dtype=float64), array([-7.00000000e-04, -1.25000000e-02, -6.95200000e-03, -1.34062400e-01,
       -1.52016000e-02, -3.78400000e-03, -2.49033000e-02, -1.56179000e-02,
       -2.69841000e-02, -2.83441000e-02, -1.62894000e-02, -4.12964000e-02,
       -1.65648000e-02, -7.50000000e-06, -2.99250000e-03, -4.80000000e-04,
       -1.06000000e-01, -5.72400000e-02, -1.22560000e-01, -1.13720000e-01,
       -3.36087500e-03, -5.27585500e-02, -1.21842500e-03, -1.62150000e-04,
       -1.00000000e-02, -6.45561000e-03, -3.27950000e-04, -3.27950000e-04,
       -1.22060089e-01, -4.70180183e-02, -2.04384070e-03, -6.51665850e-03,
       -1.65960080e-03])], 'isotopes': [array([], dtype=float64), array(['6000.03c', '27059.03c', '24050.03c', '24052.03c', '24053.03c',
       '24054.03c', '42092.03c', '42094.03c', '42095.03c', '42096.03c',
       '42097.03c',

## Updating properties to materials

1. The properties must be from the following list: ['cp', 'cv', 'g', 'h', 'my', 'pr', 'r', 's', 'tc', 'v']

In [10]:
mat1.addproperty(['g'])
print(mat1)

TypeError: isinstance() arg 2 must be a type or tuple of types